In [162]:
import torch
import pandas as pd
from graphdatascience import GraphDataScience
from torch_geometric.data import Data
import torch.nn.functional as F

In [163]:
from src.shared.database_wrapper import DatabaseWrapper
from src.model.GAT.gat_encoder import GATEncoder
from src.model.GAT.gat_decoder import GATDecoder
from src.shared.graph_schema import NodeType, EdgeType, AuthorEdge, PublicationEdge

In [164]:
db = DatabaseWrapper()

2024-07-29 16:49:13,078 - DatabaseWrapper - INFO - Connecting to the database ...
2024-07-29 16:49:13,107 - DatabaseWrapper - INFO - Database ready.


In [165]:
node_to_one_hot = {
    NodeType.PUBLICATION.value: F.one_hot(torch.tensor(0), 7),
    NodeType.AUTHOR.value: F.one_hot(torch.tensor(1), 7),
    NodeType.CO_AUTHOR.value: F.one_hot(torch.tensor(2), 7),
    NodeType.ORGANIZATION.value: F.one_hot(torch.tensor(3), 7),
    NodeType.VENUE.value: F.one_hot(torch.tensor(4), 7)
}
def project_single(n):
    return torch.vstack((node_to_one_hot[n.labels], torch.tensor(n['vec'])))

def project_pub_title_and_abstract(nodes):
    pass

projection_map = {
    NodeType.PUBLICATION.value: project_single,
    NodeType.AUTHOR.value: project_single,
    NodeType.CO_AUTHOR.value: project_single,
    NodeType.ORGANIZATION.value: project_single,
    NodeType.VENUE.value: project_single
}

def convert_to_pyg(nodes, relationships):
    # Create a mapping from Neo4j node IDs to consecutive integers
    node_id_mapping = {node['id']: i for i, node in enumerate(nodes)}
    for n in nodes:
        print(list(n.labels)[0])
        print(projection_map[list(n.labels)[0]](n))
    node_features = [projection_map[list(n.labels)[0]](n) for n in nodes]

    x = torch.tensor(node_features, dtype=torch.float)

    # Create edge index
    edge_index = []
    for rel in relationships:
        source = node_id_mapping[rel.start_node.id]
        target = node_id_mapping[rel.end_node.id]
        edge_index.append([source, target])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Create PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index)

    return data


In [166]:
"""for nodes in db.iter_nodes(NodeType.PUBLICATION, ["id"]):
    for node in nodes:
        nodes, rels = db.fetch_neighborhood(NodeType.PUBLICATION, node["id"], 5)
        for n in nodes:
            print(n)
            #print(n.labels, 'Publication' in n.labels)
        print("")
        for rel in rels:
            print(rel)
        break
"""


'for nodes in db.iter_nodes(NodeType.PUBLICATION, ["id"]):\n    for node in nodes:\n        nodes, rels = db.fetch_neighborhood(NodeType.PUBLICATION, node["id"], 5)\n        for n in nodes:\n            print(n)\n            #print(n.labels, \'Publication\' in n.labels)\n        print("")\n        for rel in rels:\n            print(rel)\n        break\n'

In [167]:
for nodes in db.iter_nodes(NodeType.PUBLICATION, ["id"]):
    for node in nodes:
        nodes, rels = db.fetch_neighborhood(NodeType.PUBLICATION, node["id"], 5)
        data = convert_to_pyg(nodes, rels)
        print(data)

Publication


KeyError: frozenset({'Publication'})